In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import random

In [2]:
def sigmoid(x):
    """
    Compute the sigmoid of x

    Arguments:
    x -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(x)
    """
    s = 1/(1+np.exp(-x))
    return s

In [143]:
def relu(x):
    """
    Compute the relu of x

    Arguments:
    x -- A scalar or numpy array of any size.

    Return:
    s -- relu(x)
    """
    s = np.maximum(0,x)
    
    return s


In [66]:
# return train_X, train_Y, test_X, test_Y, val_X, val_Y
def load_dataset():
    scaled_train_dir = "scaled_chest_xray/train/"
    scaled_test_dir = "scaled_chest_xray/test/"

    scaled_train_pneu = os.listdir(os.path.join(scaled_train_dir, 'PNEUMONIA'))
    scaled_train_normal = os.listdir(os.path.join(scaled_train_dir, 'NORMAL'))
    scaled_test_pneu = os.listdir(os.path.join(scaled_test_dir, 'PNEUMONIA'))
    scaled_test_normal = os.listdir(os.path.join(scaled_test_dir, 'NORMAL'))

    scaled_train = [('PNEUMONIA/' + name, 1) for name in scaled_train_pneu] + [('NORMAL/' + name, 0) for name in scaled_train_normal]
    scaled_test = [('PNEUMONIA/' + name, 1) for name in scaled_test_pneu] + [('NORMAL/' + name, 0) for name in scaled_test_normal]

    random.shuffle(scaled_train) 
    random.shuffle(scaled_test) 

    # data = np.array(list(Image.open("scaled_chest_xray/train/NORMAL/NORMAL-28501-0001.jpeg").getdata()))
    # print(len(data))
    # print(data)
    # print(type(data))
    
    test_X_list = [list(Image.open(scaled_test_dir + image_path).getdata().convert('L')) for image_path, i in scaled_test]
    test_X_list = [[float(value)/255 for value in image_data] for image_data in test_X_list]

    train_X_list = [list(Image.open(scaled_train_dir + image_path).getdata().convert('L')) for image_path, i in scaled_train]
    train_X_list = [[float(value)/255 for value in image_data] for image_data in train_X_list]

    train_X = np.array(train_X_list, dtype=float).T
    test_X = np.array(test_X_list, dtype=float).T
    train_Y = np.array([[float(i) for image_path, i in scaled_train]], dtype=float)
    test_Y = np.array([[float(i) for image_path, i in scaled_test]], dtype=float)

    return (train_X, train_Y, test_X, test_Y)

In [67]:
train_X, train_Y, test_X, test_Y = load_dataset()
#print(train_X[0:3])
#print(train_Y)
print(test_X)
print(test_Y)
print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

[[0.         0.27843137 0.00392157 ... 0.00784314 0.00392157 0.04313725]
 [0.         0.39607843 0.         ... 0.         0.         0.04313725]
 [0.         0.36862745 0.         ... 0.         0.         0.04313725]
 ...
 [0.         0.03921569 0.00392157 ... 0.         0.         0.04313725]
 [0.01176471 0.         0.         ... 0.         0.         0.04313725]
 [0.         0.         0.         ... 0.         0.         0.04313725]]
(50176, 5232)
(1, 5232)
(50176, 624)
(1, 624)


In [68]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(2) # we set up a seed so that your output matches ours although the initialization is random.
    
    ### START CODE HERE ### (≈ 5 lines of code)
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y,1))
    parameters = {"W1":W1, "b1":b1, "W2":W2, "b2":b2}
    ### END CODE HERE ###
    
    assert (parameters["W1"].shape == (n_h, n_x))
    assert (parameters["b1"].shape == (n_h, 1))
    assert (parameters["W2"].shape == (n_y, n_h))
    assert (parameters["b2"].shape == (n_y, 1))
     
    return parameters

In [144]:
def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    # And implement Forward Propagation to calculate A2 (probabilities)

    ### START CODE HERE ### (≈ 8 lines of code)
    W1 = parameters["W1"]
    #print(W1.shape)
    b1 = parameters["b1"]
    #print(b1.shape)
    W2 = parameters["W2"]
    #print(W2.shape)
    b2 = parameters["b2"]
    #print(b2.shape)

    Z1 = np.matmul(W1, X) + b1
    #print(Z1.shape)
    A1 = relu(Z1) ## !! CHOOSE AN ACTIVATION FUNCTION
    #print(A1.shape)
    Z2 = np.matmul(W2, A1) + b2
    #print(Z2.shape)
    A2 = sigmoid(Z2) ## !! CHOOSE AN ACTIVITAION FUNCTION
    #print(A2.shape)
    ### END CODE HERE ###
    
    assert(A2.shape == (1, X.shape[1]))
    
    # Values needed in the backpropagation are stored in "cache". This will be given as an input to the backpropagation
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [145]:
def compute_cost(A2, Y):
    """
    Computes the cross-entropy cost given in equation (13)
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    cost -- cross-entropy cost given equation (13)
    
    """
    
    m = Y.shape[1] # number of example

    # Compute the cross-entropy cost
    ### START CODE HERE ### (≈ 3 lines of code)
    cost = -(1/m)* float(np.dot(Y, np.log(A2.T)) + np.dot(1-Y,np.log(1-A2.T)))
    ### END CODE HERE ###

    assert(isinstance(cost, float))

    return cost

In [146]:
def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation using the instructions above.
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    # Retrieve also A1 and A2 from dictionary "cache".
    # Backward propagation: calculate dW1, db1, dW2, db2. 

    ### START CODE HERE ### (≈ 3 (retrieve) + 6 (back prop) lines of code)
    A1 = cache["A1"]        #(4,3)
    A2 = cache["A2"]        #(1,3)
    W2 = parameters["W2"]   #(1,4)

    dZ2 = A2 - Y                                        #(1,3)
    dW2 = (1/m)*np.matmul(dZ2,A1.T)                     #(1,4)
    db2 = (1/m)*np.sum(dZ2, axis=1, keepdims = True)    #(1,1)

    #dZ1 = np.matmul(W2.T,dZ2)* (1-np.power(A1, 2))      #(4,3) # !!! CHOOSE CORRECT DERRIVATIVE FOR THE ACTIVATION FUNCTION IN THE 2. LAYER
    dZ1 = np.matmul(W2.T,dZ2)* np.heaviside(A1, 1)      #(4,3)
    dW1 = (1/m)*np.matmul(dZ1,X.T)                      #(4,2)
    db1 = (1/m)*np.sum(dZ1, axis=1, keepdims = True)    #(4,1)
    ### END CODE HERE ###
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [147]:
def update_parameters(parameters, grads, learning_rate):
    """
    Updates parameters using the gradient descent update rule given above
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve each parameter from the dictionary "parameters" and "grads"
    # Update rule for each parameter
    ### START CODE HERE ### (≈ 12 lines of code)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]

    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2

    ### END CODE HERE ###
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [148]:
def nn_model(X, Y, n_h, learning_rate, num_iterations = 10000, print_cost=False):
    n_x = X.shape[0]
    n_y = Y.shape[0]
    costs=[]

    # Initialize parameters
    # Loop (gradient descent)
    # Print every 1000 th cost to console, e.g. using print("Cost after iteration {}: {:.2e}".format(i, cost))
    ### START CODE HERE ### (≈ 12 lines of code)
    parameters = initialize_parameters(n_x, n_h, n_y)
    #return parameters
    for i in range(num_iterations):
        A2, cache = forward_propagation(X, parameters)
        cost = compute_cost(A2, Y)
        costs.append(cost)
        if print_cost == True:
            if (i+1) % 1 == 0:
                print("Cost after iteration {}: {:.2e}".format(i+1, cost))
        grads = backward_propagation(parameters, cache, X, Y)
        parameters = update_parameters(parameters, grads, learning_rate)

    ### END CODE HERE ###

    # Returns parameters learnt by the model. They can then be used to predict output
    return (parameters, costs)

In [149]:
def predict(parameters, X):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    ### START CODE HERE ### (≈ 2 lines of code)
    A2, cache = forward_propagation(X, parameters)
    predictions = np.heaviside(A2-0.5, 1)
    ### END CODE HERE ###
    
    return predictions

In [150]:
n_x = train_X.shape[0]
n_y = train_Y.shape[0]
n_h = 100 # Number of neurons per layer
learning_rate = 0.005

# Build a model with a n_h-dimensional hidden layer
parameters, costs = nn_model(train_X, train_Y, n_h, learning_rate, num_iterations = 600, print_cost=True)

plt.plot(costs)
plt.ylabel('cost')
plt.xlabel('iterations (x1,000)')
plt.title("Learning rate =" + str(learning_rate))
plt.show()

Cost after iteration 1: 6.71e-01
Cost after iteration 2: 6.49e-01
Cost after iteration 3: 6.29e-01
Cost after iteration 4: 6.13e-01
Cost after iteration 5: 6.00e-01
Cost after iteration 6: 5.89e-01
Cost after iteration 7: 5.82e-01
Cost after iteration 8: 5.77e-01
Cost after iteration 9: 5.73e-01
Cost after iteration 10: 5.70e-01
Cost after iteration 11: 5.68e-01
Cost after iteration 12: 5.67e-01
Cost after iteration 13: 5.66e-01
Cost after iteration 14: 5.65e-01
Cost after iteration 15: 5.65e-01
Cost after iteration 16: 5.64e-01
Cost after iteration 17: 5.64e-01
Cost after iteration 18: 5.63e-01
Cost after iteration 19: 5.63e-01
Cost after iteration 20: 5.62e-01
Cost after iteration 21: 5.61e-01
Cost after iteration 22: 5.61e-01
Cost after iteration 23: 5.60e-01
Cost after iteration 24: 5.60e-01
Cost after iteration 25: 5.59e-01
Cost after iteration 26: 5.59e-01
Cost after iteration 27: 5.58e-01
Cost after iteration 28: 5.58e-01
Cost after iteration 29: 5.57e-01
Cost after iteration 30

KeyboardInterrupt: 

In [ ]:
predictions = predict(parameters, train_X)
print ('Accuracy: %d' % float((np.dot(train_Y,predictions.T) + np.dot(1-train_Y,1-predictions.T))/float(train_Y.size)*100) + '%')

Accuracy: 95%


In [ ]:
predictions = predict(parameters, test_X)
print ('Accuracy: %d' % float((np.dot(test_Y,predictions.T) + np.dot(1-test_Y,1-predictions.T))/float(test_Y.size)*100) + '%')

Accuracy: 82%
